# Converting a torch model to ONNX
This notebook shows how we can transform PyTorch models into the ONNX format so we can use our model in the browser.

First, imagine we load the model from somewhere. Below, we use the potsawee model, but in the future we will use our own trained model.

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")

c:\Users\mashu\miniconda3\envs\PBE_AQG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Next we export the model to onnx. There are several things we have to do to make this work.
1. We must generate sample inputs for the model using the Tokenizer. According to [the documentation](https://pytorch.org/docs/stable/onnx.html), this is done because to convert the model to the ONNX format, the model needs to be run once with its proper input.
2. We should also provide the input and output names for columns.

In [2]:
tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")

sample_text = "In the beginning, God created the heavens and the earth."
inputs = tokenizer(sample_text, return_tensors="pt")
decoder_input_ids = torch.ones_like(inputs["input_ids"])
outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], decoder_input_ids=decoder_input_ids)
question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)

onnx_path = "onnxModel/potsawee.onnx"
input_names = ["input_ids", "attention_mask"]
output_names = ["output"]
dynamic_axes = {"input_ids": {0: "batch_size", 1: "seq_len"}, "attention_mask": {0: "batch_size", 1: "seq_len"}, "output": {0: "batch_size", 1: "seq_len"}}


c:\Users\mashu\miniconda3\envs\PBE_AQG\Lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Next we use the `torch.onnx.export` function to export the model.

In [3]:
print("inputs:", inputs)

inputs: {'input_ids': tensor([[  86,    8, 1849,    6,  601,  990,    8, 9922,    7,   11,    8, 3596,
            5,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [4]:
torch.onnx.export(model, args=tuple(inputs.values()) + (decoder_input_ids,), f=onnx_path, input_names=input_names, output_names=output_names, dynamic_axes=dynamic_axes)

c:\Users\mashu\miniconda3\envs\PBE_AQG\Lib\site-packages\transformers\modeling_utils.py:828: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:


============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



And that should be it! There should be a `potsawee.onnx` file in the `onnxModel` directory now. You can also see hundreds of `onnx_*` files that are generated. These are part of the model. Don't erase them.